In [ ]:
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt

from red_utils import split_rel_results

In [ ]:
res_df = 'res_df.2458099.43869.ee.pkl'

In [ ]:
with open(res_df.rsplit('.', 1)[0] + '.md.pkl', 'rb') as f:
    md = pickle.load(f)
    
df = pd.read_pickle(res_df)
df

In [ ]:
# check results for a given frequency & time integration
test_freq = 300
test_tint = 35

resx = df.loc[(test_freq, test_tint)][5:].values.astype(float)
test_vis, test_gains = split_rel_results(resx, md['no_unq_bls'])
test_amps = np.abs(test_gains)
test_phases = np.angle(test_gains)
mean_test_amps = np.mean(test_amps)
mean_test_phases = np.mean(test_phases)

print('Gain amplitude solutions for frequency channel {} and time integration {} are:\n{}\n'\
      .format(test_freq, test_tint, test_amps))
print('Gain phase solutions are:\n{}\n'\
      .format(test_phases))
print('Gain amplitude average is: {}'.format(mean_test_amps))
print('Gain phase average is: {}'.format(mean_test_phases))

In [ ]:
# success rate percentage
success_pct = df['success'].sum() / df['success'].size * 100
print('{}% of iterations (freq/tint slice) were succesful'.format(round(success_pct, 2)))

### Number of iterations required

In [ ]:
ylab_dict = {'nit': 'number of iterations', 'fun': 'log-likelihood'}

In [ ]:
def plot_res(df, ycol, logy=False, ylim=None, figsize=(12,8)):
    """Plot attribute of calibration results
    
    :param df: Results dataframe
    :type df: DataFrame
    :param ycol: Attribute to plot
    :type ycol: str
    :param logy: Bool to make y-axis scale logarithmic
    :type logy: bool
    :param figsize: Figure size of plot
    :type figsize: tuple
    """
    ax = df[ycol].plot(figsize=figsize, ylim=ylim)
    ylog = ''
    if logy:
        ax.set_yscale('log')
        ylog = 'Log '
    ax.set_ylabel((ylog+ylab_dict[ycol]).capitalize())
    plt.show()

In [ ]:
def plot_res_grouped(df, ycol, group_by='success', logy=False, figsize=(12,8)):
    """Plot attribute of calibration results, grouped by another attribute
    
    :param df: Results dataframe
    :type df: DataFrame
    :param ycol: Attribute to plot
    :type ycol: str
    :param group_by: Column by which to group scatter points
    :type group_by: str 
    :param logy: Bool to make y-axis scale logarithmic
    :type logy: bool
    :param figsize: Figure size of plot
    :type figsize: tuple
    """
    idx_dict = {k:i for i, k in enumerate(df.index.values)}
    
    x1 = [idx_dict[i] for i in df[df[group_by]][ycol].index.values]
    y1 = df[df[group_by]][ycol].values

    x2 = [idx_dict[i] for i in df[~df[group_by]][ycol].index.values]
    y2 = df[~df[group_by]][ycol].values

    fig, ax = plt.subplots(figsize=figsize)
    ax.scatter(x1, y1, s=4, alpha=0.5, label=group_by)
    ax.scatter(x2, y2, s=4, color='orange', zorder=1, label='~'+group_by)
    ylog = ''
    if logy:
        ax.set_yscale('log')
        ylog = 'Log '
    ax.set_ylabel((ylog+ylab_dict[ycol]).capitalize())
    plt.legend()
    plt.show()
    
    print('Max {} for minimizations with {}=False: {}\n'.format(ylab_dict[ycol], \
          group_by, round(np.max(df['fun'][~df['success']].values), 3)))

    print('Max {} for minimizations with {}=True: {}'.format(ylab_dict[ycol], \
          group_by, round(np.max(df['fun'][df['success']].values), 3)))

In [ ]:
plot_res(df, 'nit')

In [ ]:
plot_res_grouped(df, 'nit', logy=True)

### Log-likelihood

In [ ]:
plot_res(df, 'fun', ylim=[-0.1, 1])

In [ ]:
plot_res_grouped(df, 'fun', logy=True)